<a href="https://colab.research.google.com/github/code130/hello-world/blob/master/ANN_Hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pip install keras-tuner

In [89]:
pip install keras_tuner

In [3]:
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
import pandas as pd
from numpy.random import seed
from sklearn.preprocessing import minmax_scale

In [4]:
import pandas as pd
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
GENOMICS_DATA_URL = '/content/drive/MyDrive/SSI Project/datasets/gene_expression/source/Breast_GSE70947.csv'
data_frame = pd.read_csv(GENOMICS_DATA_URL)


data_frame = data_frame.drop(columns="samples")
#display(data_frame)

In [36]:
y = data_frame['type']
#display(y)

df_x = data_frame.iloc[:,2:].values
#display(df_x)

In [37]:
df_x_scaled = minmax_scale(df_x, axis = 0)

In [38]:
# define the number of features
ncol = df_x_scaled.shape[1]
ncol

35980

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(df_x_scaled, y, train_size = 0.8, random_state = seed(2052))
display(Y_train)

55                    normal
37                    normal
170    breast_adenocarcinoma
199    breast_adenocarcinoma
108                   normal
               ...          
192    breast_adenocarcinoma
197    breast_adenocarcinoma
166    breast_adenocarcinoma
0                     normal
205    breast_adenocarcinoma
Name: type, Length: 231, dtype: object

In [9]:
input_df = Input(shape = (ncol, ))

In [40]:
# Model configuration

batch_size = 256
no_epochs = 5
validation_split = 0.2

**Predict using Artificial Neural Network**

In [41]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#print(Y_train)
label_encoder = LabelEncoder()
Y_train_enc = label_encoder.fit_transform(Y_train)
Y_test_enc = label_encoder.fit_transform(Y_test)
#print(Y_train_enc)




In [42]:
X_train.shape[1:]

(35980,)

In [84]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from keras_tuner import HyperModel
from tensorflow.keras.models import Sequential

from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import time

LOG_DIR = f"{int(time.time())}"

def build_model(hp):
    model = keras.models.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
      model.add(layers.Dense(units=hp.Int('units_'+str(i), min_value=32, max_value=1024, step=32), activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),loss = 'mean_squared_error', metrics = ['accuracy'])

    return model

#**def build(hp):
      #**model = keras.models.Sequential()  
      # Tune the number of units in the first Dense layer
      # Choose an optimal value between 32-512
      #hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
      #model.add(Dense(75, input_dim =input_shape, units = hp_units, activation = 'relu'))
      #**model.add(Dense(hp.Int("input_units",min_value= 32, max_value = 256,step = 32), input_shape=X_train.shape[1:], activation='relu', ))

      #**for i in range(hp.Int("n_layers",1,4)):
        #**model.add(Dense(hp.Int(f"conv_{i}_units",min_value= 32, max_value = 256,step = 32), activation='relu' ))

      #model.add(Dense(25, activation='relu'))
      #model.add(Dense(10, activation='relu'))      
      #model.add(Dropout(rate=hp.Float('dropout_2',min_value=0.0,max_value=0.5,default=0.25,step=0.05,)))
      #model.add(Dense(units=hp.Int( 'units',min_value=32,max_value=512,step=32,default=128 )))
      #activation=hp.Choice('dense_activation',values=['relu', 'tanh', 'sigmoid'],default='relu')))
      #model.add(Dropout(rate=hp.Float( 'dropout_3',min_value=0.0,max_value=0.5,default=0.25,step=0.05)))
      #**model.add(Dense(1, activation='sigmoid'))

      # Tune the learning rate for the optimizer 
      # Choose an optimal value from 0.01, 0.001, or 0.0001
      #hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
      
      #model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])
      #**model.compile(optimizer = 'adam',loss = 'mean_squared_error', metrics = ['accuracy'])
      
      #model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

      #**return model



In [ ]:
#model = build()

In [ ]:
#model.fit(X_train_enc, Y_train_enc, batch_size=64, epochs=5, validation_data =(X_test_enc, Y_test_enc) )

In [76]:
from keras_tuner.tuners import RandomSearch

NUM_CLASSES = 10  

HYPERBAND_MAX_EPOCHS = 40
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 3
SEED=seed(2052)
INPUT_SHAPE = input_df

tuner = RandomSearch(build_model,objective='val_accuracy',seed=SEED,max_trials=MAX_TRIALS,executions_per_trial=EXECUTION_PER_TRIAL,directory=LOG_DIR)

In [77]:
tuner.search(X_train, Y_train_enc, epochs =5, batch_size=64, validation_data=(X_test, Y_test_enc))

Trial 20 Complete [00h 00m 14s]
val_accuracy: 0.8390804727872213

Best val_accuracy So Far: 0.8793103496233622
Total elapsed time: 00h 04m 39s
INFO:tensorflow:Oracle triggered exit


In [79]:
tuner.results_summary()

Results summary
Results in 1627004469/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 3
units_0: 64
units_1: 256
learning_rate: 0.0001
units_2: 256
units_3: 480
units_4: 320
units_5: 64
units_6: 320
units_7: 32
units_8: 64
units_9: 384
units_10: 256
units_11: 416
units_12: 160
units_13: 384
units_14: 448
units_15: 96
units_16: 64
units_17: 480
Score: 0.8793103496233622
Trial summary
Hyperparameters:
num_layers: 9
units_0: 352
units_1: 352
learning_rate: 0.0001
units_2: 512
units_3: 192
units_4: 32
units_5: 416
units_6: 224
units_7: 512
units_8: 288
units_9: 448
units_10: 352
units_11: 416
units_12: 32
units_13: 416
units_14: 288
units_15: 448
units_16: 448
units_17: 128
Score: 0.8793103496233622
Trial summary
Hyperparameters:
num_layers: 13
units_0: 352
units_1: 320
learning_rate: 0.0001
units_2: 224
units_3: 224
units_4: 384
units_5: 352
units_6: 256
units_7: 416
units_8: 416
units_9: 320
units_10: 256


In [47]:
#tuner.search_space_summary()
tuner.get_best_hyperparameters()

In [48]:

# Which was the best model?
best_model = tuner.get_best_models()

# What were the best hyperparameters?
best_hyperparameters = tuner.get_best_hyperparameters() 


In [82]:
from keras_tuner import Hyperband


bandtuner = Hyperband(
    build_model,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL
)

In [83]:
bandtuner.search(X_train, Y_train_enc, epochs =5, batch_size=64, validation_data=(X_test, Y_test_enc))

Trial 90 Complete [00h 02m 07s]
val_accuracy: 0.5229884882767996

Best val_accuracy So Far: 0.982758641242981
Total elapsed time: 00h 27m 35s
INFO:tensorflow:Oracle triggered exit


In [95]:
bandtuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 3
units_0: 64
units_1: 256
learning_rate: 0.0001
units_2: 256
units_3: 480
units_4: 320
units_5: 64
units_6: 320
units_7: 32
units_8: 64
units_9: 384
units_10: 256
units_11: 416
units_12: 160
units_13: 384
units_14: 448
units_15: 96
units_16: 64
units_17: 480
units_18: 352
tuner/epochs: 40
tuner/initial_epoch: 14
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 6b79c07442d0c1858095cf16378c3c7b
Score: 0.982758641242981
Trial summary
Hyperparameters:
num_layers: 5
units_0: 224
units_1: 288
learning_rate: 0.0001
units_2: 320
units_3: 288
units_4: 256
units_5: 352
units_6: 192
units_7: 416
units_8: 352
units_9: 448
units_10: 352
units_11: 288
units_12: 192
units_13: 96
units_14: 192
units_15: 288
units_16: 160
units_17: 64
units_18: 384
tuner/epochs: 40
tuner/initial_epoch: 14
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: e89d0d7ef

In [ ]:
#display(Y_train)